In [1]:
#匯入所需模組

import os
import pandas as pd
import numpy as np
import cv2
import PIL.Image
from keras.preprocessing import image
from keras.utils import np_utils
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.applications import *
from keras.optimizers import Adam
from keras.layers import Dense, Activation, Flatten
from keras.layers import Input,Dropout
from keras.models import Model

Using TensorFlow backend.


In [2]:
label_list=list()
img_name_list=list()
fp=open('./new_train.csv', "r")
for line in iter(fp):
    img_name_list.append(line[:-3])
    if line[-2:-1]=='A':
        label_list.append(0)
    elif line[-2:-1]=='B':
        label_list.append(1)
    elif line[-2:-1]=='C':
        label_list.append(2)
fp.close()

In [3]:
dev_label_list=list()
dev_img_name_list=list()
fp=open('./dev.csv', "r")
for line in iter(fp):
    dev_img_name_list.append(line[:-3])
    if line[-2:-1]=='A':
        dev_label_list.append(0)
    elif line[-2:-1]=='B':
        dev_label_list.append(1)
    elif line[-2:-1]=='C':
        dev_label_list.append(2)
fp.close()

In [4]:
#讀進訓練集並做正規化
path='./new_C1-P1_Train/'
img_data_list=[]

for img in img_name_list:
    img_path=path+img
    IMG=image.load_img(img_path,target_size=(224,224))
    im=image.img_to_array(IMG)
    im=im/255.0
    img_data_list.append(im)

img_data = np.array(img_data_list)

print (img_data.shape)  

(22396, 224, 224, 3)


In [5]:
path='./C1-P1_Dev/'
img_data_list=[]

for img in dev_img_name_list:
    img_path=path+img
    IMG=image.load_img(img_path,target_size=(224,224))
    im=image.img_to_array(IMG)
    im=im/255.0
    img_data_list.append(im)

dev_img_data = np.array(img_data_list)

print (dev_img_data.shape)  

(800, 224, 224, 3)


In [6]:
#設定標籤值
num_classes=3
Y_train=np_utils.to_categorical(label_list,num_classes)
x_train,y_train=shuffle(img_data,Y_train,random_state=2)

Y_test=np_utils.to_categorical(dev_label_list,num_classes)
x_test,y_test=shuffle(dev_img_data,Y_test,random_state=2)

# #訓練集80%、驗證集20%
# x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=2)

In [7]:
#使用InceptionV3進行模型訓練
image_input=Input(shape=(224,224,3))
model=VGG16(include_top=False, weights='imagenet', input_shape=(224,224,3),input_tensor=image_input)

last_layer = model.output
last_layer=Flatten()(last_layer)
last_layer=Dropout(0.5)(last_layer)
out = Dense(num_classes, activation='softmax', name='softmax')(last_layer)
custom_V3_model = Model(image_input, out)
# custom_V3_model.summary()

custom_V3_model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=1e-5),metrics=['accuracy'])
custom_V3_model.fit(x_train,y_train,validation_data=(x_test,y_test),batch_size=16,epochs=30,verbose=1)







Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 22396 samples, validate on 800 samples
Epoch 1/30
22396/22396 [==============================] - 724s 32ms/step - loss: 0.7410 - acc: 0.6660 - val_loss: 0.6227 - val_acc: 0.7163
Epoch 2/30
22396/22396 [==============================] - 667s 30ms/step - loss: 0.5682 - acc: 0.7534 - val_loss: 0.5388 - val_acc: 0.7562
Epoch 3/30
22396/22396 [==============================] - 668s 30ms/step - loss: 0.5054 - acc: 0.7812 - val_loss: 0.5518 - val_acc: 0.7475
Epoch 4/30
22396/22396 [==============================] - 661s 30ms/step - loss: 0.4487 - acc: 0.8081 - val_loss: 0.5500 - val_acc: 0.7638
Epoch 5/30
22396/22396 [==============================] - 662s 30ms/step - loss: 0.3778 - acc: 0.8455 - val_loss: 0.5450 - val_acc: 0.7937
Epoch 6/30
22396/22396 [=================